In [32]:
from chameleon.pipeline.enhanced_rag_pipeline import EnhancedRAGPipeline
from chameleon.base import PipelineConfig, RetrieverConfig, GeneratorConfig, MemoryConfig
from chameleon.preprocessing.query_transformer import QueryTransformer
from chameleon.postprocessing.contextual_compressor import ContextualCompressor
from chameleon.memory.entity_memory import EntityMemory
from chameleon.utils.logging_utils import setup_colored_logger, COLORS
from setup import PipelineFactory

from langchain_core.tools import Tool
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

import time
import datetime
from typing import List, Dict, Any

from dotenv import load_dotenv

load_dotenv()

True

## Dataset Loading : Test Datas

In [33]:
# Load African History dataset
def load_sample_documents(max_docs: int = 20) -> List[Document]:
    """Load sample documents for demonstration."""
    documents = PipelineFactory.load_test_data_from_dataset(max_documents=max_docs)
    print(f"📚 Loaded {len(documents)} document chunks")
    return documents

# Load documents
documents = load_sample_documents(max_docs=15)

05:02:33 - Loaded 208 document chunks from 15 source documents


📚 Loaded 208 document chunks


## Tools Definition

In [34]:
def create_web_search_tool():
    """Create a simulated web search tool."""
    def search_web(query: str) -> str:
        current_date = datetime.datetime.now().strftime("%Y-%m-%d")
        return f"🌐 Web search results for '{query}' ({current_date}):\n" + \
               "• Recent developments in this area\n" + \
               "• Multiple authoritative sources found\n" + \
               "• Latest research indicates significant progress"
    
    return Tool(
        name="web_search",
        description="Search for recent information not in documents",
        func=search_web
    )

def create_calculator_tool():
    """Create a simple calculator tool."""
    def calculate(expression: str) -> str:
        try:
            result = eval(expression)
            return f"🧮 Calculation: {expression} = {result}"
        except:
            return f"❌ Invalid expression: {expression}"
    
    return Tool(
        name="calculator",
        description="Perform mathematical calculations",
        func=calculate
    )

# Create tools
tools = [create_web_search_tool(), create_calculator_tool()]

## Basic RAG Pipeline

In [35]:
def create_basic_rag_pipeline(documents: List[Document]) -> Any:
    """Create a basic RAG pipeline."""
    print("🔧 Creating Basic RAG Pipeline...")
    
    pipeline = PipelineFactory.create_pipeline(
        documents=documents,
        title="Basic RAG Demo",
        rag_type="modular",
        memory_type="buffer",
        retriever_config={
            "top_k": 3,
            "similarity_threshold": 0.3
        },
        generator_config={
            "temperature": 0.7,
            "max_tokens": 1000,
            "model_name": "gpt-4.1",
            "provider": "openai"
        }
    )
    
    print("✅ Basic RAG Pipeline created!")
    return pipeline

# Create basic pipeline
basic_pipeline = create_basic_rag_pipeline(documents)

05:04:09 - Created buffer memory for pipeline
05:04:09 - Processing 208 documents in batches to avoid token limits


🔧 Creating Basic RAG Pipeline...


05:04:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:04:10 - Loading faiss.
05:04:10 - Successfully loaded faiss.
05:04:10 - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
05:04:10 - Processing batch 1/4 (50 documents)
05:04:11 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:04:11 - Processing batch 2/4 (50 documents)
05:04:12 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:04:12 - Processing batch 3/4 (50 documents)
05:04:13 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:04:13 - Processing batch 4/4 (8 documents)
05:04:14 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:04:14 - Created faiss vector store with text-embedding-3-small embeddings
05:04:14 - Created retriever with faiss vector store with text-emb

✅ Basic RAG Pipeline created!


### Enhanced RAG Pipeline

In [57]:
def create_enhanced_rag_pipeline(documents: List[Document]) -> EnhancedRAGPipeline:
    """Create an enhanced RAG pipeline with advanced features."""
    print("🚀 Creating Enhanced RAG Pipeline...")
    
    # Configuration
    config = PipelineConfig(
        rag_type="enhanced",
        retriever_config=RetrieverConfig(
            top_k=5,
            similarity_threshold=0.3,
            retrieval_type="semantic",
            reranking_enabled=True,
            filtering_enabled=True,
            filtering_threshold=0.2,
            multi_query_enabled=True,
            embedding_model="all-MiniLM-L6-v2",
            parent_document_enabled=False
        ),
        generator_config=GeneratorConfig(
            temperature=0.7,
            max_tokens=1500,
            model_name="gpt-4.1-mini",
            provider="openai"
        ),
        memory_config=MemoryConfig(
            memory_type="entity",
            max_history=10
        ),
        chain_type="stuff"
    )
    
    # Create postprocessors
    contextual_compressor = ContextualCompressor(
        compression_mode="paragraph",
        min_relevance_score=0.7
    )
    
    # Create enhanced pipeline
    pipeline = EnhancedRAGPipeline(
        title="Enhanced RAG Demo",
        documents=documents,
        config=config,
        preprocessors=[],
        postprocessors=[contextual_compressor],
        tools=tools,
        enable_evaluation=False
    )
    
    print("✅ Enhanced RAG Pipeline created!")
    return pipeline

# Create enhanced pipeline
enhanced_pipeline = create_enhanced_rag_pipeline(documents)

🚀 Creating Enhanced RAG Pipeline...


05:20:15 - Use pytorch device: mps
05:20:15 - Use pytorch device_name: mps
05:20:15 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/svngoku/Documents/global_projects/Python/agents/CHAMELON-RAG/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Enhanced RAG Pipeline created!


### Test basic pipeline

In [55]:
def test_pipeline(pipeline, queries: List[str], pipeline_name: str = "Pipeline"):
    """Test a pipeline with multiple queries."""
    print(f"\n🔍 Testing {pipeline_name}")
    print("=" * 60)
    
    results = []
    total_time = 0
    
    for i, query in enumerate(queries, 1):
        print(f"\n📝 Query {i}: {query}")
        
        try:
            start_time = time.time()
            
            # Use appropriate run method based on pipeline type
            if hasattr(pipeline, 'run') and len(pipeline.run.__code__.co_varnames) == 2:
                # Enhanced pipeline (only takes query)
                response = pipeline.run(query)
            else:
                # Basic pipeline (takes query and documents)
                response = pipeline.run(query)
            
            query_time = time.time() - start_time
            total_time += query_time
            
            print(f"⚡ Time: {query_time:.2f}s")
            print(f"🤖 Response: {response['response'][:300]}...")
            
            if 'metadata' in response:
                metadata = response['metadata']
                if 'num_docs' in metadata:
                    print(f"📊 Documents used: {metadata['num_docs']}")
            
            results.append({
                'query': query,
                'response': response,
                'time': query_time,
                'success': True
            })
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            results.append({
                'query': query,
                'error': str(e),
                'time': 0,
                'success': False
            })
    
    # Summary
    successful = [r for r in results if r['success']]
    print(f"\n📈 {pipeline_name} Summary:")
    print(f"  • Successful queries: {len(successful)}/{len(queries)}")
    if successful:
        avg_time = total_time / len(successful)
        print(f"  • Average time: {avg_time:.2f}s")
        print(f"  • Total time: {total_time:.2f}s")
    
    return results

## Querying the pipeline

In [46]:
# Define test queries
african_history_queries = [
    "What were the major kingdoms in ancient Africa?",
    "How did iron production influence African societies?",
    "What role did trade play in African empires?",
    "Describe the Aksumite kingdom and its significance",
    "What were the characteristics of Neolithic cultures in Africa?"
]

rag_technical_queries = [
    "What are the advantages of modular RAG systems?",
    "How does retrieval-augmented generation work?",
    "What are the key components of a RAG pipeline?",
    "Compare different RAG architectures",
    "How do you optimize RAG performance?"
]

### Running the basic pipeline

In [47]:
# Test basic pipeline
print("🔧 BASIC RAG PIPELINE TEST")
basic_results = test_pipeline(basic_pipeline, african_history_queries[:3], "Basic RAG")

05:14:06 - Running modular RAG pipeline: Basic RAG Demo
05:14:06 - Query: What were the major kingdoms in ancient Africa?


🔧 BASIC RAG PIPELINE TEST

🔍 Testing Basic RAG

📝 Query 1: What were the major kingdoms in ancient Africa?


05:14:07 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:14:07 - Generating response for query: What were the major kingdoms in ancient Africa?...
05:14:07 - Using 3 documents for context
05:14:16 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:14:16 - Successfully generated response
05:14:16 - Running modular RAG pipeline: Basic RAG Demo
05:14:16 - Query: How did iron production influence African societies?


⚡ Time: 9.71s
🤖 Response: Ancient Africa was home to many influential kingdoms and states, each with its own unique history and contributions. Some of the major kingdoms in ancient Africa include:

**1. Ancient Egypt (Dynastic Egypt)**
- Location: Northeastern Africa, along the Nile River (modern Egypt)
- Time period: c. 3,0...
📊 Documents used: 3

📝 Query 2: How did iron production influence African societies?


05:14:16 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:14:16 - Generating response for query: How did iron production influence African societies?...
05:14:16 - Using 3 documents for context
05:14:27 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:14:27 - Successfully generated response
05:14:27 - Running modular RAG pipeline: Basic RAG Demo
05:14:27 - Query: What role did trade play in African empires?


⚡ Time: 10.94s
🤖 Response: Iron production had a profound and transformative impact on African societies. Here are some key ways it influenced social, economic, and political development:

1. **Technological Advancement and Agriculture**
   - The adoption of iron tools (such as hoes, axes, and ploughs) revolutionized agricult...
📊 Documents used: 3

📝 Query 3: What role did trade play in African empires?


05:14:27 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
05:14:27 - Generating response for query: What role did trade play in African empires?...
05:14:27 - Using 3 documents for context
05:14:33 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:14:33 - Successfully generated response


⚡ Time: 6.19s
🤖 Response: Trade played a crucial role in the rise, prosperity, and influence of African empires. Here are some key ways in which trade impacted these societies:

1. Economic Growth and Wealth: Trade brought significant wealth to African empires. By exchanging regional products—such as gold, ivory, camwood (us...
📊 Documents used: 3

📈 Basic RAG Summary:
  • Successful queries: 3/3
  • Average time: 8.95s
  • Total time: 26.84s


### Running the Enhanced RAG Pipeline

In [59]:
# Test enhanced pipeline
print("\n🚀 ENHANCED RAG PIPELINE TEST")
enhanced_results = test_pipeline(enhanced_pipeline, african_history_queries, "Enhanced RAG")

05:21:18 - Running enhanced RAG pipeline: Enhanced RAG Demo
05:21:18 - Query: What were the major kingdoms in ancient Africa?



🚀 ENHANCED RAG PIPELINE TEST

🔍 Testing Enhanced RAG

📝 Query 1: What were the major kingdoms in ancient Africa?


05:21:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:21:20 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:21:20 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

05:22:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:38 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:38 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:39 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:39 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:40 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:41 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:41 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:42 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:42 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
05:22:43 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

⚡ Time: 27.65s
🤖 Response: Neolithic cultures in Africa, beginning around 12,000 years ago, exhibited a wide range of developments that emerged independently across different regions. Key characteristics include:

1. **Domestication and Agriculture**: Plants and animals were domesticated. In West Africa, pottery, livestock, a...
📊 Documents used: 2

📈 Enhanced RAG Summary:
  • Successful queries: 5/5
  • Average time: 19.60s
  • Total time: 98.02s


### Evaluation

In [60]:
def analyze_performance(basic_results: List[Dict], enhanced_results: List[Dict]):
    """Analyze and compare pipeline performance."""
    import matplotlib.pyplot as plt
    
    # Extract successful results
    basic_success = [r for r in basic_results if r['success']]
    enhanced_success = [r for r in enhanced_results if r['success']]
    
    if not basic_success or not enhanced_success:
        print("❌ Insufficient data for comparison")
        return
    
    # Calculate metrics
    basic_times = [r['time'] for r in basic_success]
    enhanced_times = [r['time'] for r in enhanced_success]
    
    basic_lengths = [len(r['response']['response']) for r in basic_success]
    enhanced_lengths = [len(r['response']['response']) for r in enhanced_success]
    
    # Display comparison
    print("📊 PERFORMANCE COMPARISON")
    print("=" * 50)
    print(f"Basic RAG:")
    print(f"  • Avg time: {sum(basic_times)/len(basic_times):.2f}s")
    print(f"  • Avg response length: {sum(basic_lengths)/len(basic_lengths):.0f} chars")
    print(f"  • Success rate: {len(basic_success)}/{len(basic_results)} ({len(basic_success)/len(basic_results)*100:.1f}%)")
    
    print(f"\nEnhanced RAG:")
    print(f"  • Avg time: {sum(enhanced_times)/len(enhanced_times):.2f}s")
    print(f"  • Avg response length: {sum(enhanced_lengths)/len(enhanced_lengths):.0f} chars")
    print(f"  • Success rate: {len(enhanced_success)}/{len(enhanced_results)} ({len(enhanced_success)/len(enhanced_results)*100:.1f}%)")

# Run analysis
analyze_performance(basic_results, enhanced_results)

📊 PERFORMANCE COMPARISON
Basic RAG:
  • Avg time: 8.95s
  • Avg response length: 2270 chars
  • Success rate: 3/3 (100.0%)

Enhanced RAG:
  • Avg time: 19.60s
  • Avg response length: 829 chars
  • Success rate: 5/5 (100.0%)
